In [1]:
import numpy as np
import matplotlib.pyplot as plt

from src.input import Input_uc, Input_ed
from src.output import Output_uc, Output_ed
from src.unit_commitment import solve_uc
from src.economic_dispatch import solve_ed

In [2]:
num_units=3
num_periods=6
num_buses=1

In [3]:
COPYPASTE = np.zeros(num_periods)

In [4]:
input_uc = Input_uc(
    # meta
    num_units=num_units,
    num_periods=num_periods,
    num_buses=num_buses,
    voll=3500*1300,
    let_blackout=False,
    curtail_penalty=0,
    let_curtail=False,
    exact_reserve=False,
    # renewable
    solar_p_max=COPYPASTE.copy(),
    solar_p_min=COPYPASTE.copy(),
    wind_p=COPYPASTE.copy(),
    hydro_p=COPYPASTE.copy(),
    # system
    load=[220, 250, 200, 170, 230, 190],
    system_reserve_up=[10, 10, 10, 10, 10, 10], # COPYPASTE.copy(),
    system_reserve_down=COPYPASTE.copy(), # [10, 10, 10, 0, 10, 10], 
    # operational constraint
    p_min=[80, 50, 30],
    p_max=[300, 200, 100],
    ramp_up=[50, 60, 70],
    ramp_down=[30, 40, 50],
    startup_ramp=[100, 70, 40],
    shutdown_ramp=[80, 50, 30],
    min_up=[3, 2, 1],
    min_down=[2, 2, 2],
    # generation cost function
    cost_quad=[0, 0, 0],
    cost_lin=[5, 15, 30],
    cost_const=[0, 0, 0],
    # previous horizon
    min_up_prev=[2, 0, 0],
    min_down_prev=[0, 0, 0],
    p_prev=[120, 0, 0],
    u_prev=[
        [1],
        [0],
        [0],
    ],
    # startup cost function
    cost_startup_step=[
        [800],
        [500],
        [250],
    ],
)

In [5]:
output_uc = Output_uc()
solve_uc(input_uc=input_uc, output_uc=output_uc)

Set parameter Username
Set parameter LicenseID to value 2659802
Academic license - for non-commercial use only - expires 2026-05-01


In [6]:
output_ed = Output_ed(num_periods=num_periods, num_units=num_units, num_buses=num_buses)

for time_period in range(num_periods):
    input_ed = Input_ed(
        # meta
        time_period=time_period,
        num_units=num_units,
        num_buses=num_buses,
        voll=3500*1300,
        let_blackout=False,
        curtail_penalty=0,
        let_curtail=False,
        exact_reserve=False,
        # renewable
        solar_p_max=COPYPASTE.copy(),
        solar_p_min=COPYPASTE.copy(),
        # uc
        input_uc=input_uc,
        output_uc=output_uc,
    )

    solve_ed(input_ed=input_ed, output_ed=output_ed)

output_ed.compute_auxiliary_results()

In [7]:
output_ed.__dict__

{'total_cost_retailor': 6300.0,
 'total_cost_system': 8050.0,
 'total_cost_generation': 8050.0,
 'total_cost_voll': 0.0,
 'total_cost_curtail_penalty': 0.0,
 'cost_retailor': array([1100., 1250., 1000.,  850., 1150.,  950.]),
 'cost_system': array([1600., 1750., 1000.,  850., 1900.,  950.]),
 'cost_generation': array([1600., 1750., 1000.,  850., 1900.,  950.]),
 'cost_voll': array([0., 0., 0., 0., 0., 0.]),
 'cost_curtail_penalty': array([0., 0., 0., 0., 0., 0.]),
 'z': array([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]]),
 'p': array([[170., 200., 200., 170., 200., 190.],
        [ 50.,  50.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,  30.,   0.]]),
 'blackout': array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]),
 'solar_p': array([0., 0., 0., 0., 0., 0.]),
 'solar_curtail': array([0., 0., 0., 0., 0., 0.]),
 'smp': array([5., 5., 5., 5., 5., 5.]),
 'cost_reserve_up': array([0., 0., 0., 0., 0., 0.]),
 'cost_

In [8]:
np.all(output_uc.p == output_ed.p)

np.True_

In [9]:
output_uc.cost_generation == output_ed.cost_generation

array([ True,  True,  True,  True,  True,  True])

In [10]:
output_uc.cost_system

array([2100., 1750., 1000.,  850., 2150.,  950.])

In [11]:
output_ed.cost_system

array([1600., 1750., 1000.,  850., 1900.,  950.])

In [12]:
output_ed.cost_voll

array([0., 0., 0., 0., 0., 0.])